In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [4]:
CONFIG_FILE_PATH

PosixPath('../config/config.yaml')

In [19]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
     params_filepath = PARAMS_FILE_PATH,
     schema_filepath = SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


In [20]:
import urllib.request as request
from src.datascience import logger
import zipfile

In [23]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config

    # zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with info: \n{headers}")
        else:
            logger.info(f"File exists:")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            


In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-07-19 11:27:39,087: INFO: common: yaml file: ../config/config.yaml loaded successfully]
[2025-07-19 11:27:39,089: INFO: common: yaml file: ../params.yaml loaded successfully]
[2025-07-19 11:27:39,090: INFO: common: yaml file: ../schema.yaml loaded successfully]
[2025-07-19 11:27:39,091: INFO: common: Created directory at: artifacts]
[2025-07-19 11:27:39,091: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-07-19 11:27:39,762: INFO: 2495662148: artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F97C:36AB51:91D55C:A8ACD9:687B7319
Accept-Ranges: bytes
Date: Sat, 19